In [0]:
from pyspark.sql.functions import col, to_date

bronze_table = "mycatalog.default.unemployment_bronze"
silver_table = "mycatalog.default.unemployment_silver"

df = spark.table(bronze_table)

unemp_silver = (
    df.filter(col("GEO") == "Alberta")
      .filter(col("Labour_force_characteristics") == "Unemployment rate")
      .filter(col("Statistics") == "Estimate")
      .select(
          col("REF_DATE").alias("date"),
          col("GEO").alias("geo"),
          col("VALUE").alias("unemployment_rate")
      )
      .withColumn("date", to_date(col("date"), "yyyy-MM-dd"))
      .withColumn("unemployment_rate", col("unemployment_rate").cast("double"))
)

unemp_silver.write.format("delta").mode("overwrite").saveAsTable(silver_table)

print("✅ unemployment_silver created successfully!")
display(spark.table(silver_table).limit(20))
